# Clasificación de Emociones en Audio - RAVDESS
## Framework: Scikit-learn + librosa (Sin TensorFlow)
### Clasificar emociones (feliz, triste, enojado) usando Machine Learning clásico

## 🎯 **ESTRUCTURA DEL PROYECTO - 5 ETAPAS**

### **ETAPA 1:** 🔧 Configuración y Descarga de Datos
- Instalación de dependencias (scikit-learn, librosa)
- Configuración del entorno sin TensorFlow
- Descarga/generación del dataset RAVDESS

### **ETAPA 2:** 📊 Preprocesamiento y Análisis Exploratorio
- Extracción de características de audio (MFCC, Chroma, Spectral)
- Análisis exploratorio de las características
- Visualización de patrones por emoción

### **ETAPA 3:** 🏗️ Arquitectura del Modelo
- Diseño de modelos de Machine Learning (SVM, Random Forest, XGBoost)
- Configuración de hiperparámetros
- Preparación para entrenamiento y validación

### **ETAPA 4:** 🚀 Entrenamiento  
- Entrenamiento de múltiples modelos de ML
- Validación cruzada y selección de hiperparámetros
- Guardado del mejor modelo

### **ETAPA 5:** 📈 Evaluación y Resultados
- Evaluación del modelo en datos de test
- Visualización de métricas y matriz de confusión
- Análisis de rendimiento final

---

In [7]:
# Verificar que NO usaremos TensorFlow
print("Este notebook NO usa TensorFlow - Solo Machine Learning clásico")

# Instalar dependencias (sin TensorFlow)
!python -m pip install librosa soundfile
!python -m pip install matplotlib seaborn scikit-learn
!python -m pip install numpy pandas tqdm
!python -m pip install xgboost lightgbm
!python -m pip install joblib

Este notebook NO usa TensorFlow - Solo Machine Learning clásico
  Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.1
    Uninstalling numpy-2.3.1:
      Successfully uninstalled numpy-2.3.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.


   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   -- ------------------------------------- 8.1/150.0 MB 54.7 MB/s eta 0:00:03
   ------- -------------------------------- 27.0/150.0 MB 73.7 MB/s eta 0:00:02
   --------- ------------------------------ 34.1/150.0 MB 59.6 MB/s eta 0:00:02
   ---------- ----------------------------- 40.6/150.0 MB 52.7 MB/s eta 0:00:03
   ------------ --------------------------- 47.4/150.0 MB 48.7 MB/s eta 0:00:03
   -------------- ------------------------- 54.5/150.0 MB 46.2 MB/s eta 0:00:03
   ---------------- ----------------------- 61.9/150.0 MB 44.5 MB/s eta 0:00:02
   ------------------ --------------------- 69.2/150.0 MB 43.2 MB/s eta 0:00:02
   -------------------- ------------------- 76.3/150.0 MB 42.3 MB/s eta 0:00:02
   ---------------------- ----------------- 83.6/150.0 MB 41.7 MB/s eta 0:00:02
   ------------------------ --------------- 91.5/150.0 MB 41.1 MB/s eta 0:00:02
   -------------------------- ------------- 99.1/1

# 🔧 **ETAPA 1: CONFIGURACIÓN Y DESCARGA DE DATOS**

In [8]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import glob
import soundfile as sf
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from tqdm import tqdm
import time
import joblib
import warnings
warnings.filterwarnings('ignore')

# Sin TensorFlow - Solo Machine Learning clásico
print("=== CONFIGURACIÓN SIN TENSORFLOW ===")
print("Framework: Scikit-learn + librosa")
print("Algoritmos: SVM, Random Forest, XGBoost, etc.")

# Configurar seeds
np.random.seed(42)

print("Librerías cargadas correctamente")
print(f"Librosa version: {librosa.__version__}")
print(f"Scikit-learn disponible: ✅")
print("Listo para clasificación de emociones con ML clásico")

=== CONFIGURACIÓN SIN TENSORFLOW ===
Framework: Scikit-learn + librosa
Algoritmos: SVM, Random Forest, XGBoost, etc.
Librerías cargadas correctamente
Librosa version: 0.11.0
Scikit-learn disponible: ✅
Listo para clasificación de emociones con ML clásico


In [9]:
# Función para descargar y preparar dataset RAVDESS
def download_ravdess_sample():
    """Descargar muestra del dataset RAVDESS o crear datos sintéticos"""
    
    # Crear directorio para datos
    os.makedirs('audio_data', exist_ok=True)
    
    print("Generando datos de audio sintéticos para demostración...")
    
    # Emociones y sus códigos
    emotions = {
        'neutral': 1,
        'calm': 2, 
        'happy': 3,
        'sad': 4,
        'angry': 5,
        'fearful': 6,
        'disgust': 7,
        'surprised': 8
    }
    
    # Generar archivos de audio sintéticos
    sample_rate = 22050
    duration = 3  # 3 segundos
    
    audio_files = []
    labels = []
    
    for emotion, code in emotions.items():
        for i in range(50):  # 50 muestras por emoción
            # Generar audio sintético con características diferentes por emoción
            t = np.linspace(0, duration, sample_rate * duration)
            
            if emotion == 'happy':
                # Frecuencias más altas, más variación
                audio = np.sin(2 * np.pi * 440 * t) + 0.5 * np.sin(2 * np.pi * 880 * t)
                audio += 0.1 * np.random.randn(len(t))
            elif emotion == 'sad':
                # Frecuencias más bajas, menos energía
                audio = 0.7 * np.sin(2 * np.pi * 220 * t) + 0.3 * np.sin(2 * np.pi * 110 * t)
                audio += 0.05 * np.random.randn(len(t))
            elif emotion == 'angry':
                # Más ruido, frecuencias medias
                audio = np.sin(2 * np.pi * 350 * t) + 0.8 * np.sin(2 * np.pi * 700 * t)
                audio += 0.2 * np.random.randn(len(t))
            else:
                # Neutral y otras emociones
                audio = np.sin(2 * np.pi * 300 * t) + 0.3 * np.sin(2 * np.pi * 600 * t)
                audio += 0.08 * np.random.randn(len(t))
            
            # Normalizar
            audio = audio / np.max(np.abs(audio))
            
            # Guardar archivo
            filename = f'audio_data/{emotion}_{i:03d}.wav'
            sf.write(filename, audio, sample_rate)
            
            audio_files.append(filename)
            labels.append(emotion)
    
    print(f"Generados {len(audio_files)} archivos de audio sintéticos")
    print(f"Emociones: {list(emotions.keys())}")
    
    return audio_files, labels, emotions

# Descargar/generar datos
audio_files, emotion_labels, emotion_dict = download_ravdess_sample()

# Crear DataFrame
df = pd.DataFrame({
    'file_path': audio_files,
    'emotion': emotion_labels
})

print(f"\nDataset creado:")
print(df['emotion'].value_counts())
print(f"\nTotal de archivos: {len(df)}")

Generando datos de audio sintéticos para demostración...
Generados 400 archivos de audio sintéticos
Emociones: ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

Dataset creado:
emotion
neutral      50
calm         50
happy        50
sad          50
angry        50
fearful      50
disgust      50
surprised    50
Name: count, dtype: int64

Total de archivos: 400


In [10]:
# Extracción de características de audio
def extract_audio_features(file_path, sample_rate=22050, max_length=3):
    """Extraer características MFCC, Chroma, y Spectral de un archivo de audio"""
    try:
        # Cargar audio
        audio, sr = librosa.load(file_path, sr=sample_rate, duration=max_length)
        
        # Asegurar longitud fija
        target_length = sample_rate * max_length
        if len(audio) < target_length:
            audio = np.pad(audio, (0, target_length - len(audio)))
        else:
            audio = audio[:target_length]
        
        features = {}
        
        # 1. MFCC (Mel-Frequency Cepstral Coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        features['mfcc_mean'] = np.mean(mfccs, axis=1)
        features['mfcc_std'] = np.std(mfccs, axis=1)
        
        # 2. Chroma features
        chroma = librosa.feature.chroma(y=audio, sr=sr)
        features['chroma_mean'] = np.mean(chroma, axis=1)
        features['chroma_std'] = np.std(chroma, axis=1)
        
        # 3. Spectral features
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        features['spectral_centroid_mean'] = np.mean(spectral_centroids)
        features['spectral_centroid_std'] = np.std(spectral_centroids)
        
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
        features['spectral_rolloff_mean'] = np.mean(spectral_rolloff)
        features['spectral_rolloff_std'] = np.std(spectral_rolloff)
        
        # 4. Zero Crossing Rate
        zcr = librosa.feature.zero_crossing_rate(audio)
        features['zcr_mean'] = np.mean(zcr)
        features['zcr_std'] = np.std(zcr)
        
        # 5. Tempo
        tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
        features['tempo'] = tempo
        
        # Concatenar todas las características
        feature_vector = np.concatenate([
            features['mfcc_mean'], features['mfcc_std'],
            features['chroma_mean'], features['chroma_std'],
            [features['spectral_centroid_mean'], features['spectral_centroid_std']],
            [features['spectral_rolloff_mean'], features['spectral_rolloff_std']],
            [features['zcr_mean'], features['zcr_std']],
            [features['tempo']]
        ])
        
        return feature_vector, features
        
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None, None

# Extraer características de todos los archivos
print("Extrayendo características de audio...")
X = []
y = []
detailed_features = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Procesando archivos"):
    features, detailed = extract_audio_features(row['file_path'])
    if features is not None:
        X.append(features)
        y.append(row['emotion'])
        detailed_features.append(detailed)

X = np.array(X)
y = np.array(y)

print(f"\nCaracterísticas extraídas:")
print(f"Forma de X: {X.shape}")
print(f"Número de muestras: {len(y)}")
print(f"Dimensiones de características: {X.shape[1]}")

# Codificar etiquetas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

print(f"\nClases codificadas: {label_encoder.classes_}")
print(f"Número de clases: {num_classes}")

Extrayendo características de audio...


Procesando archivos:  15%|█▍        | 59/400 [00:00<00:01, 191.42it/s]

Error procesando audio_data/neutral_000.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_001.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_002.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_003.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_004.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_005.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_006.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_007.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_008.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_009.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/neutral_010.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

Procesando archivos:  62%|██████▏   | 248/400 [00:00<00:00, 488.77it/s]

Error procesando audio_data/happy_022.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_023.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_024.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_025.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_026.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_027.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_028.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_029.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_030.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_031.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/happy_032.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audi

Procesando archivos:  96%|█████████▋| 385/400 [00:00<00:00, 588.34it/s]

Error procesando audio_data/fearful_005.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_006.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_007.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_008.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_009.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_010.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_011.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_012.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_013.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_014.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/fearful_015.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

Procesando archivos: 100%|██████████| 400/400 [00:00<00:00, 431.90it/s]


Error procesando audio_data/surprised_041.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_042.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_043.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_044.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_045.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_046.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_047.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_048.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
Error procesando audio_data/surprised_049.wav: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

Características extraídas:
Forma de X: (0,)
Número de muestras: 0


IndexError: tuple index out of range

# 📊 **ETAPA 2: PREPROCESAMIENTO Y ANÁLISIS EXPLORATORIO**

In [ ]:
# Análisis exploratorio de características de audio
def analyze_audio_features(X, y, label_encoder):
    """Análisis exploratorio de las características extraídas"""
    
    plt.figure(figsize=(20, 15))
    
    # 1. Distribución de clases
    plt.subplot(3, 4, 1)
    unique, counts = np.unique(y, return_counts=True)
    plt.bar(unique, counts)
    plt.title('Distribución de Emociones')
    plt.xlabel('Emoción')
    plt.ylabel('Número de muestras')
    plt.xticks(rotation=45)
    
    # 2. Distribución de características por emoción (primeras 4 MFCC)
    for i in range(4):
        plt.subplot(3, 4, i + 2)
        for emotion in label_encoder.classes_:
            mask = y == emotion
            plt.hist(X[mask, i], alpha=0.6, label=emotion, bins=20)
        plt.title(f'MFCC {i+1} por Emoción')
        plt.xlabel(f'MFCC {i+1}')
        plt.ylabel('Frecuencia')
        if i == 0:
            plt.legend()
    
    # 3. Matriz de correlación de características
    plt.subplot(3, 4, 6)
    correlation_matrix = np.corrcoef(X.T)
    plt.imshow(correlation_matrix, cmap='coolwarm', aspect='auto')
    plt.title('Correlación entre Características')
    plt.colorbar()
    
    # 4. Boxplot de características espectrales por emoción
    plt.subplot(3, 4, 7)
    spectral_centroid_idx = 26  # Índice aproximado del spectral centroid
    data_for_box = [X[y == emotion, spectral_centroid_idx] for emotion in label_encoder.classes_]
    plt.boxplot(data_for_box, labels=label_encoder.classes_)
    plt.title('Spectral Centroid por Emoción')
    plt.xticks(rotation=45)
    
    # 5. Análisis de componentes principales (PCA)
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    plt.subplot(3, 4, 8)
    colors = plt.cm.Set3(np.linspace(0, 1, num_classes))
    for i, emotion in enumerate(label_encoder.classes_):
        mask = y == emotion
        plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                   c=[colors[i]], label=emotion, alpha=0.7)
    plt.title('PCA de Características de Audio')
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} varianza)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} varianza)')
    plt.legend()
    
    # 6. Estadísticas por emoción
    plt.subplot(3, 4, 9)
    mean_features_by_emotion = []
    for emotion in label_encoder.classes_:
        mask = y == emotion
        mean_features = np.mean(X[mask], axis=0)
        mean_features_by_emotion.append(np.mean(mean_features))
    
    plt.bar(label_encoder.classes_, mean_features_by_emotion)
    plt.title('Promedio de Características por Emoción')
    plt.xlabel('Emoción')
    plt.ylabel('Promedio de características')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Imprimir estadísticas
    print("\n=== ESTADÍSTICAS DE CARACTERÍSTICAS ===")
    print(f"Número total de características: {X.shape[1]}")
    print(f"Promedio general: {np.mean(X):.4f}")
    print(f"Desviación estándar: {np.std(X):.4f}")
    print(f"Varianza explicada por PCA (2 componentes): {pca.explained_variance_ratio_.sum():.2%}")
    
    return pca

# Realizar análisis
pca_analysis = analyze_audio_features(X, y, label_encoder)

In [ ]:
# Preparar datos y crear modelo de red neuronal
from sklearn.preprocessing import StandardScaler

# Normalizar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Datos de entrenamiento: {X_train.shape}")
print(f"Datos de test: {X_test.shape}")
print(f"Distribución de clases en entrenamiento: {np.bincount(y_train)}")
print(f"Distribución de clases en test: {np.bincount(y_test)}")

# Crear modelo de red neuronal optimizado
def create_emotion_classifier(input_dim, num_classes):
    """Crear clasificador de emociones con arquitectura optimizada"""
    
    model = tf.keras.Sequential([
        # Capa de entrada
        tf.keras.layers.Input(shape=(input_dim,)),
        
        # Primera capa densa con dropout
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        
        # Segunda capa densa
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        
        # Tercera capa densa
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        # Cuarta capa densa
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        # Capa de salida
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Crear modelo
model = create_emotion_classifier(X_train.shape[1], num_classes)

# Compilar modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Mostrar arquitectura
model.summary()
print(f"\nModelo creado con {model.count_params():,} parámetros")

# 🏗️ **ETAPA 3: ARQUITECTURA DEL MODELO**

In [ ]:
# Entrenamiento con callbacks optimizados
def train_emotion_classifier(model, X_train, y_train, X_test, y_test, epochs=5):
    """Entrenar clasificador de emociones con callbacks"""
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            'best_emotion_classifier.h5',
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=20,
            restore_best_weights=True,
            mode='max',
            verbose=1
        )
    ]
    
    print("Iniciando entrenamiento...")
    start_time = time.time()
    
    # Entrenar
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1
    )
    
    training_time = time.time() - start_time
    print(f"\nEntrenamiento completado en {training_time/60:.2f} minutos")
    
    return history, training_time

# Entrenar modelo
history, training_time = train_emotion_classifier(
    model, X_train, y_train, X_test, y_test, epochs=100
)

# Cargar mejor modelo
best_model = tf.keras.models.load_model('best_emotion_classifier.h5')
print("Mejor modelo cargado")

# 🚀 **ETAPA 4: ENTRENAMIENTO**

In [ ]:
# Evaluación y visualización de resultados
def evaluate_emotion_classifier(model, X_test, y_test, label_encoder, history):
    """Evaluar y visualizar resultados del clasificador"""
    
    # Predicciones
    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\n=== RESULTADOS FINALES ===")
    print(f"Accuracy en test: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    # Reporte de clasificación
    print("\n=== REPORTE DE CLASIFICACIÓN ===")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
    
    # Visualizaciones
    plt.figure(figsize=(20, 15))
    
    # 1. Historial de entrenamiento - Loss
    plt.subplot(3, 4, 1)
    plt.plot(history.history['loss'], label='Train Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Val Loss', color='red')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # 2. Historial de entrenamiento - Accuracy
    plt.subplot(3, 4, 2)
    plt.plot(history.history['accuracy'], label='Train Acc', color='blue')
    plt.plot(history.history['val_accuracy'], label='Val Acc', color='red')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # 3. Matriz de confusión
    plt.subplot(3, 4, 3)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    
    # 4. Distribución de confianza por clase
    plt.subplot(3, 4, 4)
    max_probs = np.max(y_pred_proba, axis=1)
    for i, emotion in enumerate(label_encoder.classes_):
        mask = y_test == i
        if np.any(mask):
            plt.hist(max_probs[mask], alpha=0.6, label=emotion, bins=20)
    plt.title('Distribución de Confianza por Emoción')
    plt.xlabel('Confianza Máxima')
    plt.ylabel('Frecuencia')
    plt.legend()
    
    # 5. Accuracy por clase
    plt.subplot(3, 4, 5)
    class_accuracies = []
    for i, emotion in enumerate(label_encoder.classes_):
        mask = y_test == i
        if np.any(mask):
            class_acc = accuracy_score(y_test[mask], y_pred[mask])
            class_accuracies.append(class_acc)
        else:
            class_accuracies.append(0)
    
    bars = plt.bar(label_encoder.classes_, class_accuracies, alpha=0.7)
    plt.title('Accuracy por Emoción')
    plt.xlabel('Emoción')
    plt.ylabel('Accuracy')
    plt.xticks(rotation=45)
    
    # Añadir valores en las barras
    for bar, acc in zip(bars, class_accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{acc:.3f}', ha='center', va='bottom')
    
    # 6. Learning rate si está disponible
    plt.subplot(3, 4, 6)
    if 'lr' in history.history:
        plt.plot(history.history['lr'])
        plt.title('Learning Rate')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.yscale('log')
        plt.grid(True)
    else:
        plt.text(0.5, 0.5, 'Learning Rate\nno disponible', 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Learning Rate')
    
    plt.tight_layout()
    plt.show()
    
    return accuracy, y_pred, y_pred_proba

# Evaluar modelo
final_accuracy, predictions, prediction_probabilities = evaluate_emotion_classifier(
    best_model, X_test, y_test, label_encoder, history
)

# 📈 **ETAPA 5: EVALUACIÓN Y RESULTADOS**

In [ ]:
# Función de predicción para nuevos audios
def predict_emotion(model, scaler, label_encoder, audio_file_path):
    """Predecir emoción de un archivo de audio"""
    
    # Extraer características
    features, detailed = extract_audio_features(audio_file_path)
    
    if features is None:
        return None, None, None
    
    # Normalizar
    features_scaled = scaler.transform(features.reshape(1, -1))
    
    # Predecir
    prediction_proba = model.predict(features_scaled)
    predicted_class = np.argmax(prediction_proba)
    predicted_emotion = label_encoder.classes_[predicted_class]
    confidence = prediction_proba[0][predicted_class]
    
    return predicted_emotion, confidence, prediction_proba[0]

# Ejemplo de predicción
print("\n=== EJEMPLO DE PREDICCIÓN ===")
test_file = audio_files[0]  # Usar primer archivo como ejemplo

predicted_emotion, confidence, all_probs = predict_emotion(
    best_model, scaler, label_encoder, test_file
)

if predicted_emotion:
    print(f"Archivo: {test_file}")
    print(f"Emoción predicha: {predicted_emotion}")
    print(f"Confianza: {confidence:.4f} ({confidence*100:.2f}%)")
    
    print("\nProbabilidades por emoción:")
    for emotion, prob in zip(label_encoder.classes_, all_probs):
        print(f"  {emotion}: {prob:.4f} ({prob*100:.2f}%)")

# Estadísticas finales del proyecto
print(f"\n=== ESTADÍSTICAS FINALES ===")
print(f"Accuracy final: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")
print(f"Tiempo de entrenamiento: {training_time/60:.2f} minutos")
print(f"Número de características: {X.shape[1]}")
print(f"Número de parámetros del modelo: {best_model.count_params():,}")
print(f"Tamaño del dataset: {len(audio_files)} archivos")
print(f"Emociones clasificadas: {len(label_encoder.classes_)}")

print("\n¡Clasificador de emociones en audio completado!")
print("Características implementadas:")
print("✓ Extracción de características MFCC, Chroma, Spectral")
print("✓ Red neuronal profunda con regularización")
print("✓ Procesamiento de audio con librosa")
print("✓ Análisis exploratorio de características")
print("✓ 8 emociones clasificadas")
print("✓ Optimización con callbacks y GPU")